In [1]:
import altair as alt
import jupyterlab_omnisci.vega_ibis
import ibis
import pandas as pd


alt.data_transformers.enable('ibis')
alt.renderers.enable('ibis')

RendererRegistry.enable('ibis')

## Simple Data

In [2]:
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
})
connection = ibis.pandas.connect({'source': source })
table = connection.table('source')

alt.Chart(table).mark_bar().encode(
    x='a',
    y='b'
)

/usr/local/miniconda3/envs/jupyterlab-omnisci/lib/python3.7/site-packages/altair/utils/core.py:90: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ))


Chart({
  data: Empty DataFrame
  Columns: [a, b]
  Index: [],
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'a'
    }),
    y: Y({
      shorthand: 'b'
    })
  }),
  mark: 'bar'
})

## Aggregate

In [3]:
jupyterlab_omnisci.vega_ibis.FALLBACK = False

In [5]:
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52],
    'c': ['first', 'first', 'first', 'second', 'second', 'second', 'third', 'third', 'third']

})
connection = ibis.pandas.connect({'source': source })
table = connection.table('source')

jupyterlab_omnisci.vega_ibis._executed_expressions = []

alt.Chart(table).mark_bar().encode(
    x='c',
    y='mean(b)'
)

Chart({
  data: Empty DataFrame
  Columns: [a, b, c]
  Index: [],
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'c'
    }),
    y: Y({
      shorthand: 'mean(b)'
    })
  }),
  mark: 'bar'
})

Verify that the only query computed is an aggregate:

In [6]:
_expr, = jupyterlab_omnisci.vega_ibis._executed_expressions
_expr

ref_0
PandasTable[table]
  name: source
  schema:
    a : string
    b : int64
    c : string

Aggregation[table]
  table:
    Table: ref_0
  metrics:
    mean_b = Mean[float64]
      b = Column[int64*] 'b' from table
        ref_0
      where:
        None
  by:
    c = Column[string*] 'c' from table
      ref_0

## Filter

In [7]:
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52],
    'c': ['first', 'first', 'first', 'second', 'second', 'second', 'third', 'third', 'third']

})
connection = ibis.pandas.connect({'source': source })
table = connection.table('source')

jupyterlab_omnisci.vega_ibis._executed_expressions = []

alt.Chart(table).mark_bar().encode(
    x='a',
    y='b'
).transform_filter(
    alt.datum.c == 'first'
)

Chart({
  data: Empty DataFrame
  Columns: [a, b, c]
  Index: [],
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'a'
    }),
    y: Y({
      shorthand: 'b'
    })
  }),
  mark: 'bar',
  transform: [FilterTransform({
    filter: (datum.c === 'first')
  })]
})

In [8]:
_expr, = jupyterlab_omnisci.vega_ibis._executed_expressions
_expr

ref_0
PandasTable[table]
  name: source
  schema:
    a : string
    b : int64
    c : string

Selection[table]
  table:
    Table: ref_0
  predicates:
    Equals[boolean*]
      left:
        c = Column[string*] 'c' from table
          ref_0
      right:
        Literal[string]
          first

## Interactive Filter

In [9]:
source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52],
    'c': ['first', 'first', 'first', 'second', 'second', 'second', 'third', 'third', 'third']

})

connection = ibis.pandas.connect({'source': source })
table = connection.table('source')

jupyterlab_omnisci.vega_ibis._executed_expressions = []


input_dropdown = alt.binding_select(options=['first','second'])
selection = alt.selection_single(fields=['c'], bind=input_dropdown, name='Filter')

alt.Chart(table).mark_bar().encode(
    x='a',
    y='b'
).add_selection(
    selection
).transform_filter(
    selection
)

Chart({
  data: Empty DataFrame
  Columns: [a, b, c]
  Index: [],
  encoding: FacetedEncoding({
    x: X({
      shorthand: 'a'
    }),
    y: Y({
      shorthand: 'b'
    })
  }),
  mark: 'bar',
  selection: {'Filter': SelectionDef({
    bind: BindRadioSelect({
      input: 'select',
      options: ['first', 'second']
    }),
    fields: ['c'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'Filter'}
  })]
})